# Importação das tabelas

In [156]:
import pymysql
from dotenv import load_dotenv
import os
import pandas as pd
import warnings
from sklearn.preprocessing import LabelEncoder
import pickle

warnings.filterwarnings("ignore")

Credenciais

In [101]:
# Carregando as variáveis de ambiente do arquivo .env
load_dotenv()

# Obtendo as credenciais do banco de dados
DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT"))
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

Funções

In [102]:
# Função para carregar dados do MySQL em um DataFrame
def carregar_tabela_em_dataframe(tabela):
    try:
        # Conectar ao banco de dados
        conexao = pymysql.connect(
            host=DB_HOST,
            port=DB_PORT,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME,
        )
        # Query para carregar os dados
        query = f"SELECT * FROM {tabela};"

        # Carregar dados diretamente em um DataFrame
        df = pd.read_sql(query, conexao)

        # Fechar a conexão
        conexao.close()

        return df
    except Exception as e:
        print("Erro ao carregar a tabela:", e)
        return None

Tabelas

In [103]:
tabela_daumau = carregar_tabela_em_dataframe("daumau")
tabela_desinstalacoes = carregar_tabela_em_dataframe("desinstalacoes")
tabela_installs = carregar_tabela_em_dataframe("installs")
tabela_ratings_reviews = carregar_tabela_em_dataframe("ratings_reviews")

# Análise das tabelas

In [104]:
tabela_daumau.head()

,appId,date,dauReal,mauReal
0,com.app.33540,2024-01-01,393961.0,3643110.0
1,com.app.33540,2024-01-02,800448.0,3643040.0
2,com.app.31679,2024-01-01,157532.0,1349262.0
3,com.app.31679,2024-01-02,313847.0,1349317.0
4,com.app.77385,2024-01-01,239768.0,1934660.0


In [105]:
tabela_desinstalacoes.head()

,appId,date,country,lang,predictionLoss
0,com.app.71740,2024-01-01,br,pt,1463
1,com.app.33540,2024-01-01,br,pt,8170
2,com.app.19544,2024-01-01,br,pt,77048
3,com.app.77385,2024-01-01,br,pt,4497
4,com.app.31679,2024-01-01,br,pt,4287


In [106]:
tabela_installs.head()

,appid,date,newinstalls
0,com.app.86367,2024-01-01,1968
1,com.app.18711,2024-01-01,1364
2,com.app.36257,2024-01-01,7511
3,com.app.88509,2024-01-01,53137
4,com.app.93131,2024-01-01,137


In [107]:
tabela_ratings_reviews.head()

,date,appid,category,ratings,daily_ratings,reviews,daily_reviews
0,2024-01-01,com.app.40009,OTHERS,63500.0,5.0,25251.0,0.0
1,2024-01-01,com.app.88516,OTHERS,2440.0,0.0,312.0,0.0
2,2024-01-01,com.app.64634,TRAVEL_AND_LOCAL,536.0,0.0,331.0,0.0
3,2024-01-01,com.app.78470,SHOPPING,97924.0,7.0,27241.0,23.0
4,2024-01-01,com.app.36433,FINANCE,678.0,0.0,414.0,0.0


Vamos fazer uma junção das tabelas usando as chaves primárias de data e identificador do aplicativo.

In [108]:
# Alterando o nome da coluna de Id
tabela_installs.rename({"appid": "appId"}, axis=1, inplace=True)
tabela_ratings_reviews.rename({"appid": "appId"}, axis=1, inplace=True)

# Alterando oi dtype das colunas de data
tabela_desinstalacoes["date"] = tabela_desinstalacoes["date"].astype("object")
tabela_installs["date"] = tabela_installs["date"].astype("object")

In [109]:
# Juntando as tabelas
df = (
    tabela_daumau.merge(tabela_installs, on=["date", "appId"], how="left")
    .merge(tabela_desinstalacoes, on=["date", "appId"], how="left")
    .merge(tabela_ratings_reviews, on=["date", "appId"], how="left")
)

In [110]:
df.head()

,appId,date,dauReal,mauReal,newinstalls,country,lang,predictionLoss,category,ratings,daily_ratings,reviews,daily_reviews
0,com.app.33540,2024-01-01,393961.0,3643110.0,NaN,NaN,NaN,NaN,FINANCE,223848.0,297.0,82172.0,46.0
1,com.app.33540,2024-01-02,800448.0,3643040.0,NaN,NaN,NaN,NaN,FINANCE,224145.0,892.0,82218.0,58.0
2,com.app.31679,2024-01-01,157532.0,1349262.0,NaN,NaN,NaN,NaN,FINANCE,277813.0,132.0,139907.0,19.0
3,com.app.31679,2024-01-02,313847.0,1349317.0,NaN,NaN,NaN,NaN,FINANCE,277945.0,303.0,139926.0,14.0
4,com.app.77385,2024-01-01,239768.0,1934660.0,NaN,NaN,NaN,NaN,FINANCE,197329.0,187.0,79637.0,25.0


Colunas presentes no dataset:

appId: Identificador único do aplicativo.

date: Data de registro.

dauReal: Usuários ativos diários.

mauReal: Usuários ativos mensais.

newInstalls: Novas instalações diárias.

country: País de origem dos dados.

lang: Idioma predominante.

predictionLoss: Erro de previsão.

category: Categoria do aplicativo.

ratings: Avaliação geral do aplicativo.

daily_ratings: Avaliações recebidas no dia.

reviews: Total de avaliações escritas.

daily_reviews: Avaliações escritas recebidas no dia.

Selecionando apenas as colunas relevantes para a criação do conjunto de dados para o treinamento do modelo. Nessa etapa, incialmente deixaremos algumas variáveis de fora devido a uma primeira análise suas características não serem úteis de imediato para prever o dauReal.

**Observação:** Cabe aqui ressaltar que poderíamos realizar algumas análises estatísticas mais aprofundadas visando eliminar mais variáveis. Porém, essa etapa ficará dedicada ao notebook 2 em que faremos uma EDA.

In [111]:
# Filtrando as variáveis
df_model = df[
    [
        "appId",
        "date",
        "newinstalls",
        "mauReal",
        "ratings",
        "daily_ratings",
        "reviews",
        "daily_reviews",
        "country",
        "category",
    ]
]

In [112]:
df_model.head()

,appId,date,newinstalls,mauReal,ratings,daily_ratings,reviews,daily_reviews,country,category
0,com.app.33540,2024-01-01,NaN,3643110.0,223848.0,297.0,82172.0,46.0,NaN,FINANCE
1,com.app.33540,2024-01-02,NaN,3643040.0,224145.0,892.0,82218.0,58.0,NaN,FINANCE
2,com.app.31679,2024-01-01,NaN,1349262.0,277813.0,132.0,139907.0,19.0,NaN,FINANCE
3,com.app.31679,2024-01-02,NaN,1349317.0,277945.0,303.0,139926.0,14.0,NaN,FINANCE
4,com.app.77385,2024-01-01,NaN,1934660.0,197329.0,187.0,79637.0,25.0,NaN,FINANCE


# Tratamento dos dados

In [113]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41299 entries, 0 to 41298
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   appId          41271 non-null  object 
 1   date           41299 non-null  object 
 2   newinstalls    0 non-null      float64
 3   mauReal        41279 non-null  float64
 4   ratings        38966 non-null  float64
 5   daily_ratings  38966 non-null  float64
 6   reviews        38966 non-null  float64
 7   daily_reviews  38966 non-null  float64
 8   country        0 non-null      object 
 9   category       38966 non-null  object 
dtypes: float64(6), object(4)
memory usage: 3.2+ MB


Aqui vemos que as colunas newinstalls e country não possuem dados preenchidos. Ou seja, teresmo que removê-las do conjunto de dados.

In [114]:
# Removendo as tabelas newinstalls e country
df_model = df_model.drop(["newinstalls", 'country'], axis = 1)

In [115]:
# Alterando o datatype da coluna de data
df_model['date'] = pd.to_datetime(df_model['date'])

# Criando colunas de ano, mes e dia
df_model['ano'] = df_model['date'].dt.year
df_model['mes'] = df_model['date'].dt.month
df_model['dia'] = df_model['date'].dt.day

In [116]:
df_model['ano'].value_counts()

ano
2024    41198
2220       42
2044       34
1912       14
1980       11
Name: count, dtype: int64

In [117]:
# Observando os dados acima filtremos somente as linhas para o ano de 2024
df_model = df_model[df_model['ano'] == 2024]

In [118]:
# vamos remover agora a coluna de data e ano que não serão úteis
df_model.drop(['date', 'ano'], axis = 1, inplace = True)

In [ ]:
# Vamos criar uma variável categórica usando o appId
le = LabelEncoder()
df_model['appId_encoded'] = le.fit_transform(df_model['appId'])

In [125]:
# Removendo a coluna de appId
df_model.drop(['appId'], axis = 1, inplace = True)

In [129]:
# Como as colunas restantes (exceto category) são numéricas e apresentam poucos valores nulos, vamos preencher com a média de cada coluna respectivamente
colunas_media = ["mauReal", "ratings", "daily_ratings", "reviews", "daily_reviews"]

for coluna in colunas_media:
    df_model[coluna] = df_model[coluna].fillna(df_model[coluna].mean())

Observa-se que apenas 5% dos dados da coluna category são nulos. Ou seja, remover tais linhas não geraria uma perda sinigficativa nos dados. Portanto, vamos remover as linhas que não possuem essa característica.

Vale ressaltar que existem inúmeras outras técnias a serem utilizadas, como preenchimento usando a moda, substituição por uma categoria específica, usar modelos de imputação como KNN ou modelos de regressão para tentar estimar a categoria faltante, etc. Mas, devido ao tempo e as características, vamos remover as linhas faltantes.

Temos embasamento para remoção pois faltam apenas 5% dos dados;

Em um projeto real, utilizaríamos as técnicas mencionadas visando possíveis melhorias na base.

In [136]:
# Removendo as linhas faltantes em category
df_model = df_model[df_model['category'].notna()]

In [138]:
# Vamos criar uma variável categórica usando o category
le2 = LabelEncoder()
df_model['category_encoded'] = le2.fit_transform(df_model['category'])

In [140]:
# Removendo a coluna category
df_model.drop(['category'], axis = 1, inplace = True)

In [142]:
df_model.head()

,mauReal,ratings,daily_ratings,reviews,daily_reviews,mes,dia,appId_encoded,category_encoded
0,3643110.0,223848.0,297.0,82172.0,46.0,1,1,46,1
1,3643040.0,224145.0,892.0,82218.0,58.0,1,2,46,1
2,1349262.0,277813.0,132.0,139907.0,19.0,1,1,44,1
3,1349317.0,277945.0,303.0,139926.0,14.0,1,2,44,1
4,1934660.0,197329.0,187.0,79637.0,25.0,1,1,131,1


# Salvando a base

In [154]:
load_dotenv()

diretorio_projeto = str(os.getenv("DIRETORIO_PROJETO"))

df_model.to_csv(diretorio_projeto + "/data/data.csv", index = False)

In [157]:
# Salvando também os artefatos de encodding
encodings = {
    "appId": le,
    "category": le2
}

with open(diretorio_projeto + '/data/encodings.pkl', 'wb') as f:
    pickle.dump(encodings, f)